# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [27]:
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.stats import weightstats as stests

In [3]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [4]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [5]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


Q1: What test is appropriate for this problem? Does CLT apply?
A1: We will use a Binom.The CLT will apply because we have a large number of samples (more than 30 ) and each observation is independent from each other. 
Q2: What are the null and alternate hypotheses?
A2: Null hypothesis: The probability for black-sounding applicants receive calls from employer equals the probability for white-sounding applicants receive call from empolyer
    Alternative hypothesis: The probability for black-sounding applicants receive calls from employer does not equal the probability for white-sounding applicants receive call from empolyer

In [36]:
w = data[data.race=='w']
b = data[data.race=='b']
w_count = len(w)
b_count = len(b)

w_received = w[w.call == 1]
b_received = b[b.call == 1]

w_received_count = len(w_received)
b_received_count = len(b_received)

w_probability_received = w_received_count/w_count
b_probability_received = b_received_count/b_count

total_probability = (w_received_count + b_received_count)/(w_count + b_count)

print('Probability of white sounding received call: ', w_probability_received)
print('Probability of black sounding received call: ', b_probability_received)
print('Probability of Total received call: ', total_probability)

Probability of white sounding received call:  0.09650924024640657
Probability of black sounding received call:  0.06447638603696099
Probability of Total received call:  0.08049281314168377


In [45]:
# Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
# frequentist approach
# margin of error ( 95% Confidence )
z = 1.96
p1 = w_probability_received
p2 = b_probability_received
sigma_p1_p2 = np.sqrt((p1*(1-p1)/w_count) + (p2*(1-p2)/b_count))

margin_of_error = z * sigma_p1_p2
print('Margin_of_Error is: ', margin_of_error)

mu_p1_p2 = p1-p2
upper_CI = mu_p1_p2 + margin_of_error
lower_CI = mu_p1_p2 - margin_of_error

print('95 % Confidence Interval is :', (lower_CI, upper_CI))

pt = total_probability
p_value = stats.binom_test(x=(w_received_count + b_received_count), n=(w_count + b_count), p=mu_p1_p2)
print('The p value is: ', p_value )

 



Margin_of_Error is:  0.015255406349886438
95 % Confidence Interval is : (0.016777447859559147, 0.047288260559332024)
The p value is:  3.6626665117682988e-59


Q4.Write a story describing the statistical significance in the context or the original problem.
A4: The p-value of this two-sample z test is much smaller than alpha value of 0.05. We will reject the null hypothesis which is an indication that mean number of calls received for white-sounding and black-sounding does not equal. From the confidence interval, we can find out that white-sounding applicants has a higher chance to receive calls. 
5.Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?
A5: Current analysis can only prove that race/name is one factor in callback success. Because we have not analyze the impact of other variables in the dataset. Other information might have a bigger impact on the callback success. We can run an analysis of variance to figure out which factor/factors make the most impact of callback success, then run the hypothesis test on those factor/factors. 